In [232]:
%matplotlib inline

# подключение библиотек
import matplotlib.pyplot as plt
import numpy as np
import copy

# формат вывода чисел
np.set_printoptions(precision=3, suppress=True, formatter={'all': lambda x: f'{x:0.6f}'}) 

In [233]:
# поиск главного элемента в матрице
def find_main_element(A, iteration):
    size = A.shape[0]
    main_element = A[iteration,iteration]
    i_main = iteration
    j_main = iteration

    for i in range(size - iteration): 
        for j in range(size - iteration):
            if abs(A[i + iteration,j + iteration]) > abs(main_element):
                i_main = i + iteration
                j_main = j + iteration
                main_element = A[i_main,j_main]
    return [i_main, j_main]

In [234]:
# Функция для обмена строк
def swap_rows(A, i, j):
    A[[i, j], :] = A[[j, i], :]

# Функция для обмена столбцов
def swap_columns(A, i, j):
    A[:, [i, j]] = A[:, [j, i]]
    
#Функция для обмена значений
def swap(a,b):
    return b,a


In [235]:
def gauss(a, f):
    
    A = copy.deepcopy(a)
    F = copy.deepcopy(f)
    
    size = len(A)  # размеры матрицы - количество переменных Х
    
    X = np.arange(size) # массив для смены порядка переменных Х
        
    for iter in range(size): # прямой ход
        
        main_indices = find_main_element(A,iter) # поиск координат главного элемента
        main_element = A[main_indices[0], main_indices[1]] # галвный элемент
        
        # если главный элемент не стоит в верхней строке, делаем перестановку строк
        if(main_indices[0] - iter != 0): 
            swap_rows(A[iter:, iter:], 0, main_indices[0] - iter)
            # swap(F[iter], F[main_indices[0]])
            F[iter] = F[main_indices[0]] + F[iter]
            F[main_indices[0]] = F[iter] - F[main_indices[0]]
            F[iter] = F[iter] - F[main_indices[0]]
            
            
        # если главный элемент не стоит в левом столбце,  делаем перестановку столбцов
        if(main_indices[1] - iter != 0): 
            swap_columns(A, iter, main_indices[1])
            # swap(X[iter], X[main_indices[1]])
            X[iter] = X[main_indices[1]] + X[iter]
            X[main_indices[1]] = X[iter] - X[main_indices[1]]
            X[iter] = X[iter] - X[main_indices[1]]
        
        # делим строчку на главный элемент
        A[iter:, iter:][0] =  A[iter:, iter:][0] / main_element #
        F[iter] = F[iter] / main_element
        
        # вычитаем из всех строк первую домноженную на соответсвующий множитель
        for i in range(size - iter - 1):
            F[iter + i + 1] -= (F[iter]  * A[iter:, iter:][i + 1][0])
            A[iter:, iter:][i + 1] -= (A[iter:, iter:][0] * A[iter:, iter:][i + 1][0])

    # приводим правый нижний элемент к 1
    F[-1] /= A[-1, -1]
    A[size - 1, size - 1] = 1
 
    U  = np.ones((size, 1)) # массив для решений СЛАУ
    
    # обратный ход
    for i in range(size-1, -1, -1):
        U[i] = F[i]
        for j in range(i + 1, size):
            U[i] -= U[j] * A[i][j]

    # перестановка переменных в изначальном порядке
    result = np.ones((size, 1))
    for i in range(size):
        result[int(X[i])][0] = U[i][0]

    # возвращаем результат
    return result



In [ ]:
a = 10
size = 100

A = np.eye(size) * (a - 1) + np.ones(size)

F = np.zeros((size, 1))
for i in range(size): F[i] = i + 1


print("A is \n", A)
print("F is \n", F)

U = gauss(A, F)

print("U is \n", U)

Res = A.dot(U)
print("A * U = \n", Res)

In [371]:
def LU(a, f):
    
    A = copy.deepcopy(a)
    F = copy.deepcopy(f)
    
    # if (is_positive(A) == -1):
    #     return

    # Размер матрицы A
    size = len(A)
    
    # Инициализируем матрицы L и U
    L = np.zeros((size, size))
    U = np.zeros((size, size))
    
    # Проходим по всем строкам и столбцам
    for i in range(size):
        # Вычисляем элементы матрицы U
        for j in range(i, size):
            U[i, j] = A[i, j] - sum(L[i, k] * U[k, j] for k in range(i))
        
        # Вычисляем элементы матрицы L
        for j in range(i, size):
            if i == j:
                L[i, i] = 1  # Диагональные элементы L равны 1
            else:
                L[j, i] = (A[j, i] - sum(L[j, k] * U[k, i] for k in range(i))) / U[i, i]

# Обратный ход для Ly = F -> ищем y

    Y = np.ones((size, 1))
            
    for i in range(0, size):
        Y[i] = F[i]
        for j in range(0, i):
            Y[i] -= Y[j] * L[i][j]

# Обратный ход для Ux = Y -> ищем x
    
    X  = np.ones((size, 1))

    for i in range(size - 1, -1, -1):
        X[i] = Y[i]
        for j in range(i + 1, size):
            X[i] -= X[j] * U[i][j]
        X[i] /= U[i][i]
        
    return X

In [ ]:
a = 10
size = 100

A = np.eye(size) * (a - 1) + np.ones(size)

F = np.zeros((size, 1))
for i in range(size): F[i] = i + 1

U = LU(A, F)

print("U is ", U)

Res = A.dot(U)
print("A * U = ", Res)

In [240]:
#Вычисление третей нормы вектора
def norm_3(U):
    r = 0
    for i in range(len(U)):
        r += U[i,0] * U[i,0]
        
    return np.sqrt(r)

In [353]:
def yakobi(a, f, x_0, eps):
    
    A = np.copy(a)
    F = np.copy(f)
    
    X_prev = np.copy(x_0)
    X_new = np.copy(x_0)
    
    size = len(A)
        
    k = 0
    r = norm_3(A @ X_new - F)
    while (r > eps):
        for i in range(size):
            sum_Ax = sum(A[i, j] * X_prev[j] for j in range(size) if j != i)
            X_new[i] = (F[i] - sum_Ax) / A[i, i]
            
        X_prev = X_new.copy()
        r = norm_3(A @ X_new - F)
        k += 1
        
    return X_new, k

In [ ]:
a = 10
size = 10


A = np.eye(size, dtype=np.float64) * (a - 1) + np.ones(size, dtype=np.float64)

F = np.zeros((size, 1), dtype=np.float64)
for i in range(size): F[i, 0] = i + 1

U_0 = np.ones((size,1), dtype=np.float64)

eps = 1e-5


print("A = \n", A)
print("F = \n", F)
print("U_0 = \n", U_0)


U, k = yakobi(A, F, U_0, eps) # ошибка в реализации Якоби, для матриц больше 10 попадает в вечный цикл и не выходит

print("U is \n", U)
print("k = ", k)
print(" def = \n", norm_3(A @ U - F))

G = gauss(A, F)
print("guass is \n",G )
print("gauss def = ", norm_3(A @ G - F))

In [363]:
def seidel(a, f, x_0, eps):
    
    A = np.copy(a)
    F = np.copy(f)
    
    X_prev = np.copy(x_0)
    X_new = np.copy(x_0)
    
    size = len(A)
        
    k = 0
    r = norm_3(A @ X_new - F)
    while(r > eps):
        for i in range(size):
            sum1 = 0
            for j in range(i):
                sum1 += A[i, j] * X_new[j]
            
            sum2 = 0
            for j in range(i + 1, size):
                sum2 += A[i, j] * X_prev[j]
            
            X_new[i] = (F[i] - sum1 - sum2) / A[i, i]

        
        X_prev = X_new.copy()
        r = norm_3(A @ X_new - F)
        k += 1
        
    return X_new, k

In [ ]:
a = 10
size = 100


A = np.eye(size, dtype=np.float64) * (a - 1) + np.ones(size, dtype=np.float64)

F = np.zeros((size, 1), dtype=np.float64)
for i in range(size): F[i, 0] = i + 1

U_0 = np.ones((size,1), dtype=np.float64)

eps = 1e-5


print("A = \n", A)
print("F = \n", F)
print("U_0 = \n", U_0)


U, k = seidel(A, F, U_0, eps)

print("U is \n", U)
print("k = ", k)
print(" def = \n", norm_3(A @ U - F))

G = gauss(A, F)
print("guass is \n",G )
print("gauss def = ", norm_3(A @ G - F))